### Trainieren eines Neurons (Klassifizierung) ![alt text](image.png)

Wir trainieren nun ein Neuron, das 0 für Mohn und 1 für Enzian ausgibt.
Um den Prozess besser nachvollziehen zu können, lassen wir es zuerst jeweils anhand von je einem Wert auswendig lernen, was Mohn und was Enzian ist.

In [ ]:

import numpy ## um Arrays und Matrizen nutzen zu können


## Trainingsdaten

breiteMohn = 1.0
laengeMohn = 0.9
pflanzeM = 0 # Mohn

inputMohn = numpy.array([breiteMohn,laengeMohn])
target = pflanzeM
print(f"Die einzelnen Merkmalswerte zu dem Tag sind in einer Liste gespeichert: {inputMohn}")         
print(f"Der Zielwert ist die Pflanzenart (0=Mohn): {target}")  

Wenn die Breite 1 ist und die Länge 0.9, dann soll das Neuron 0 ausgeben. 
Dazu müssen die Gewichte die Input-Werte entsprechend anpassen. Es geht also darum, die richtigen Gewichte zu bestimmen.

In [ ]:
# Die Gewichte sind anfangs i.d.R. zufällig festgelegt 
# Hier sind es feste Zahlenwerte, um reproduzierbare Ergebnisse zu erzielen.
weights = numpy.array([0.3, 0.5])


```mermaid
flowchart LR
    I1((Breite<br>1</br>))
    I2((Länge<br>0.9</br>))
    N1((Pflanze))
    I1 --0.3--> N1
    I2 --0.5--> N1
```

In [ ]:
# Vorwärtsrechnung Perzeptron, forward propagation (Aktivierungsfunktion: lineare Funktion)
# einzelne Rechnungen:
output = weights[0] * inputMohn[0]  +  weights[1] * inputMohn[1]
print(f"Erste Schätzung: {output}. Wir erwarten {target}")

Es gibt also noch eine Abweichung vom Erwarteten Wert.
Nun bestimmt man den Fehler, der gemacht wurde, um anhand des Fehlers die Gewichte zu korrigieren (Rückwärtsrechnung, Backpropagation)

In [ ]:
# Berechnung des Fehlers
error = target - output
print(f"Das KI-System hat einen Fehler von {error} gemacht")
# Berechnung des quadratischen Fehlers (quadratisch --> stetiges Minimierungsproblem)
# Der Einfachheit halber ist die Fehlerfunktion hier nur für einen einzelnen Output definiert.
MSE = numpy.square(target - output)   
print(f"Die Fehlerfunktion Mean Square Error ergibt {MSE}. Diese soll minimiert werden.")

$𝑡$: true value, tatsächlicher Wert

$𝑜$: Output, Ausgabewert der KI

$MSE$: Mittlerer quadratischer Fehler bzw. Mean Squared Error (MSE)

$MSE=(𝑡−𝑜)^2$

Dieser Fehler wird verwendet, um die Gewichte anzupassen.
Dazu muss die Ableitung des Fehlers bezogen auf jeweils das Gewicht $w_1$ und $w_2$ bestimmt werden.
Diese Ableitung nach einer bestimmten Größe nennt man partielle Ableitung und wird statt mit einem $d$ mit einem $\partial$ angegeben.

Wir berechnen also, um wie viel das Gewicht $w_1$ verändert werden muss durch:

$\Delta w_0=\frac{\partial MSE}{\partial w_0}=\frac{\partial}{\partial w_0}\cdot(t-o)^2$

nach Anwendung der Kettenregel ergibt das:

$\Delta w_0=2\cdot(t-o)\cdot\frac{\partial}{\partial w_0}\cdot(t-o)$

Mit $o=w_0\cdot i_0 + w_1\cdot i_1$

$\Delta w_0=2\cdot(t-o)\cdot\frac{\partial}{\partial w_0}\cdot(t- (w_0\cdot i_0 + w_1\cdot i_1))$

Da wir nur nach $w_1$ ableiten, sind die Beiträge von $w_1$ zum Ouput $o$ egal und werden weggelassen, $w_1\cdot i_1=0$ für diesen Fall. 

Wenn wir die Änderung $\Delta w_1$ bestimmen, dann muss der Betrag von $w_1$ weggelassen werden, also dann $w_0\cdot i_0 = 0$.

$\Delta w_0=2\cdot(t-o)\cdot (-i_0)$

mit $e=t-o$ (kann auch negativ werden) ergibt das

$\Delta w_0=-2\cdot e\cdot i_0$

Je größer also der Fehler und der Input, desto größer ist die Gewichtsanpassung.

Die Berechnung für $\Delta w_1$ geht genauso und sparen wir uns daher.

In [ ]:
# Berechnung der Ableitung nach dem Gewicht w0
dw0 = -2 * inputMohn[0] * error
print(f"Die Ableitung der Fehlerfunktion nach dem Gewicht w0 ergibt: {dw0}.")
# Berechnung der Ableitungen nach den Gewichten w0, w1
dw = -2*inputMohn*error
print(f"Alle partiellen Ableitungen der Fehlerfunktion in einem Vektor: {dw}.")


Nun werden die Gewichte entsprechend angepasst:


In [ ]:
print(f"Gewichte vorher: {weights}")
# Um die Gewichtswerte anzupassen müssen wir sehr kleine Schritte gehen, sonst schießen wir über das Ziel hinaus!
# Die Lernrate alpha ist ein Hyperparameter, mit dem die Schrittweite angepasst werden kann.
alpha = 1e-5    # 1e-5 = 0.000001
weights = weights - alpha * dw


print(f"Nach Korrektur:  {weights}")

```mermaid
flowchart LR
    I1((Breite<br>1</br>))
    I2((Länge<br>0.9</br>))
    N1((Pflanze))
    I1 --0.299985--> N1
    I2 --0.4999865--> N1
```

In [ ]:
# Vorher war der Fehler bei -0.75
# Mal sehen wie groß der Fehler nun nach der Anpassung ist!
output = numpy.dot(weights, inputMohn)   # das ist das gleiche wie weights[0]*input[0] + weights[1]*input[1], nur als Funktion
# Berechnung des Fehlers
error = target - output
print(f"Das KI-System hat einen Fehler von {error} gemacht, also ein kleines bisschen besser.")

Um den Fehler immer weiter zu minimieren, müssen wir den Lernprozess häufiger durchführen.
Dazu brauchen wir noch eine Abbruchbedingung, ab wann und der Fehler klein genug ist.
Wir wählen hier willkürlich 0.01

In [ ]:
zaehler = 0  #Zählt die Durchläufe
# Wir gehen in einer Schleife viele kleine Schritte, bis wir zu einem zufriedenstellenden Ergebnis gekommen sind.
# Hier soll das System lernen, bis der Fehler kleiner 0.01 ist:
while abs(error) > 0.01:    # Wird der Sollfehler zu klein eingestellt (z.B. auf 0), kann das im Allgemeinen zu einer Endlosschleife führen
    zaehler += 1
    output = numpy.dot(weights, inputMohn)
    error = target - output
    weights = weights + alpha * 2 * inputMohn * error

print(f"Wir haben {zaehler} Durchläufe benötigt.")
print(f"Die Gewichtswerte lauten jetzt {weights}")
print(f"Der Output ist jetzt {output}.")

Nun hat unser System gelernt, dass bei einem Input von Breite=1 und Länge=0.9 es sich um Mohn handelt

```mermaid
flowchart LR
    I1((Breite<br>1</br>))
    I2((Länge<br>0.9</br>))
    N1((Pflanze, <br>0.00999=Mohn</br>))
    I1 -- -0.10884009--> N1
    I2 --0.13204392--> N1
```

Was passiert nun, wenn wir ihm Daten von Enzian geben?

In [ ]:
breiteEnzian = 0.3
laengeEnzian = 0.7
pflanzeE = 1 # Enzian

inputEnzian = numpy.array([breiteEnzian, laengeEnzian])

output = numpy.dot(weights, inputEnzian)
print(f"Vorhergesagt {output}.")
print(f"Erwartet haben wir 1.00 (=Enzian).")

Wir müssen also dem Neuron noch beibringen, wann etwas enzian ist.
Dazu müssen wir es weiter trainieren.
Dazu wechseln wir in der Lernschleife zwischen den beiden Datensätzen hin und her.

In [ ]:
errorM = numpy.inf    # Fehler am Anfang unendlich groß
errorE = numpy.inf    # Fehler am Anfang unendlich groß
alpha = 1e-5           # Hyperparameter Lernrate
zaehler = 0            # Zähler für Lernschritte
targetM = pflanzeM
targetE = pflanzeE

while abs(errorM) > 0.01 and abs(errorE) > 0.01:    # Beide Fehler sollen niedrig werden
    zaehler += 2
    # Ein Lernschritt mit Datensatz input_1
    output = numpy.dot(weights, inputMohn)
    errorM = targetM - output
    weights = weights + alpha * 2 * inputMohn * errorM
    # Ein Lernschritt mit Datensatz input_2
    output = numpy.dot(weights, inputEnzian)
    errorE = targetE - output
    weights = weights + alpha * 2 * inputEnzian * errorE


print(f"Wir haben {zaehler} Durchläufe benötigt.")
print(f"Die Gewichtswerte lauten jetzt {weights}\n")
print(f"Für Mohn {numpy.dot(weights, inputMohn)}.")
print(f"Für Enzian {numpy.dot(weights, inputEnzian)}.")


```mermaid
flowchart LR
    I1((Breite<br>1</br>))
    I2((Länge<br>0.9</br>))
    N1((Pflanze))
    I1 -- -2.03781097--> N1
    I2 --2.2753455--> N1
```

Nun hat das Neuron die beiden Breiten und Längen für die beiden Klassen auswendig gelernt.

Versuchen wir es auf unbekannte Daten anzuwenden:

In [ ]:
# Daten aus NN1.ipynb
breiteEnzian = [0.3, 0.9, 0.2, 0.4, 0.6]
laengeEnzian = [0.7, 0.2, 0.5, 0.2, 0.3]

breiteMohn = [1.0, 0.8, 1.2, 0.6, 1.3]
laengeMohn = [0.9, 0.6, 0.5, 0.8, 0.7]

inputMNeu = numpy.array([breiteMohn[2],laengeMohn[2]])
print(f"Für unbekannte Mohn-Daten {numpy.dot(weights, inputMNeu)}. Erwartet ist 1 (=Mohn).")

Das Auswendiglernen der wenigen Daten bringt für unbekannte Daten nicht viel.

Wir müssen das Neuron mit mehr Daten trainieren.



In [ ]:
target = numpy.array([1,1,1,1,1,0,0,0,0,0])  # 5 mal Enzian und dann 5 mal Mohn

# Wir müssen nun darauf achten, dass die Input-Daten genau in dieser Reihenfolgen vorliegen

input = numpy.array([
    [breiteEnzian[0],laengeEnzian[0]],
    [breiteEnzian[1],laengeEnzian[1]],
    [breiteEnzian[2],laengeEnzian[2]],
    [breiteEnzian[3],laengeEnzian[3]],
    [breiteEnzian[4],laengeEnzian[4]],
    [breiteMohn[0],laengeMohn[0]],
    [breiteMohn[1],laengeMohn[1]],
    [breiteMohn[2],laengeMohn[2]],
    [breiteMohn[3],laengeMohn[3]],
    [breiteMohn[4],laengeMohn[4]],
    ])


print(f"Der Datensatz in der Zeile 3 lautet beispeilsweise {input[3-1,:]}. Der zugehörige Zielwert ist {target[3-1]}.")


In [ ]:
# Startwerte
error = numpy.inf        # Fehler am Anfang unendlich groß
alpha = 1e-5             # Hyperparameter Lernrate
anzahl_epochen = 200     # Zähler für Lernschritte
    
for zaehler in range(anzahl_epochen):               # Vorgabe einer festen Anzahl an Epochen für das Training
    for nr in range(len(input)):                    # len(input) gibt die Anzahl der Trainingsdaten an
        output = numpy.dot(weights, input[nr,:])    # input[nr,:] sind alle Inputdaten zum Datensatz mit der Zeilennummer 'nr'
        error = target[nr] - output                 # target[nr] ist der Zielwert zum Datensatz mit der Nummer 'nr'
        weights = weights + alpha * 2 * input[nr,:] * error

print(f"Wir haben {anzahl_epochen} Epochen für das Training verwendet.")
print(f"Die Gewichtswerte lauten jetzt {numpy.round(weights,3)}\n")
print(f"Für Enzian {numpy.dot(weights, input[0,:])}. Tatsächlich soll es {target[0]} sein.")
print(f"Für Mohn {numpy.dot(weights, input[1,:])}. Tatsächlich soll es {target[-1]} sein.")


Nur ein Neuron kann reicht nicht aus. Wir brauchen mehr Neuronen, die wir zu einem Neuronalen Netz vernetzen.

Dazu gibt es fertige Bibliotheken, wie z.B. tensorflow